In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
# constants
out_dir = "data/cleaned"

In [3]:
def get_int_from_polygolf_results(s):
	return int(s[1:-1])

def get_str_from_polygolf_results(s):
	return str(s[2:-2])

In [4]:
def format(src_file, out_file):
	df = pd.read_csv(src_file)

	useful_columns = [ 'player_names', 'map', 'skills', 'scores', 'player_states', 'penalties', 'timeout_count', 'error_count', 'total_time_sorted' ]
	df = df.filter(items=useful_columns)

	S, F = 0, 1
	def map_polygolf_result_row(r):
		player = get_str_from_polygolf_results(r.player_names)
		tournament_map =str(r["map"])
		skills = get_int_from_polygolf_results(r.skills)
		scores = get_int_from_polygolf_results(r.scores)
		player_states = 0 if get_str_from_polygolf_results(r.player_states) == 'S' else 1
		penalties = get_int_from_polygolf_results(r.penalties)
		timeouts = get_int_from_polygolf_results(r.timeout_count)
		errors = get_int_from_polygolf_results(r.error_count)
		time = float(r.total_time_sorted.split(',')[1][:-2])

		return pd.Series([player, tournament_map, skills, scores, player_states, penalties, timeouts, errors, time])

	column_rename_map = {k: v for k, v in enumerate(useful_columns)}
	df = df.apply(map_polygolf_result_row, axis=1).rename(columns=column_rename_map)
	df.to_csv(out_file, index=False)

In [5]:
# format Joe's tournament results data
src_file = "data/cleaned/tournament_results.csv"
out_file = f"{out_dir}/tournament_results_short_formatted.csv"
format(src_file, out_file)

In [6]:
# format mac rerun results and merge with tournament results
src_m1_file = "data/cleaned/tournament_g1_m1_rerun_results.csv"
out_m1_file = f"{out_dir}/tournament_g1_m1_short_formatted.csv"
format(src_m1_file, out_m1_file)

In [ ]:
player = 'Group 1'
rerun_config = {
	"2021_maps/g2/spider_par6.json": [10],
	"2021_maps/g7/complex2.json": [10, 25, 50],
	"maps/default/checkers2.json": [50, 100],
	"maps/g1/tournament.json": [10, 25],
	"maps/g2/mwc_swirly.json": [10, 25, 50, 100],
	"maps/g3/g3_tournament.json": [10],
	"maps/g4/chevuoi.json": [10],
	"maps/g4/jump.json": [10, 25, 50],
	"maps/g4/n.json": [10, 25, 50, 100],
	"maps/g5/tourney.json": [10, 25, 50, 100],
	"maps/g7/g7_final.json": [10],
	"maps/g8/risk_map.json": [10, 25, 50],
	"maps/g9/rhino_karma.json": [10, 25, 50, 100]
}

tournament_results = pd.read_csv(out_file)

for tmap, skills in rerun_config.items():
	condition = (tournament_results['player_names'] != player) | (tournament_results['map'] != tmap)
	skill_condition = None
	for skill in skills:
		if skill_condition is None:
			skill_condition = (tournament_results['skills'] != skill)
		else:
			skill_condition = skill_condition & (tournament_results['skills'] != skill)

	condition = condition | skill_condition

	tournament_results = tournament_results[condition]


checks = [
	(tournament_results['map'] == 'maps/g7/g7_final.json') & (tournament_results['player_names'] == player),
	(tournament_results['map'] == 'maps/g8/risk_map.json') & (tournament_results['player_names'] == player),
	(tournament_results['map'] == 'maps/g9/rhino_karma.json') & (tournament_results['player_names'] == player)
]
for check in checks:
	# print(tournament_results[check])
	pass

rerun_m1_results = pd.read_csv(out_m1_file)

In [ ]:
out_tournament_m1_file = 'data/cleaned/tournament_m1_results_short_formatted.csv'
m1_results = pd.concat([tournament_results, rerun_m1_results])
m1_results.to_csv(out_tournament_m1_file, index=False)